In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path='/content/drive/My Drive/Colab Notebooks/Data_Mining/Toxic Comment Analysis/'

In [4]:
"""Un-Comment For Running locally out of colab
path='./dataset'
"""#Un-Comment For Running locally out of colab 

"Un-Comment For Running locally out of colab\npath='./dataset'\n"

In [5]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
labels = pd.read_csv(path + "test_labels.csv")

In [7]:
train = train[:100000]
classes = list(train.columns)[2:]
classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [8]:
links = '(http://.*?\s)|(http://.*)'
ip_addr = '\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'
users = '\[\[User.*'
newline = '\\n'
def clean(comment):
  import re
  comment=comment.lower()
  comment=re.sub(links,'',comment)
  comment=re.sub(ip_addr,'',comment)
  comment=re.sub(users,'',comment)
  comment=re.sub(newline,'',comment)
  return comment

In [9]:
train['comment_text']=train['comment_text'].map(lambda i:clean(i))
test['comment_text']=test['comment_text'].map(lambda i:clean(i))

In [10]:
x=train['comment_text']
y=train.iloc[:,2:8]
print(x.shape)
print(y.shape)

(100000,)
(100000, 6)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [12]:
labels = pd.read_csv(path + "test_labels.csv")
labels=labels.iloc[:,1:]
sum_labels=np.sum(labels,axis=1)
idx=sum_labels>=0

In [13]:
labels_consider=labels[idx]
test=test.iloc[:,1:]
tests_consider=test[idx].values[:,0]

print(labels_consider.shape, tests_consider.shape)

(63978, 6) (63978,)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=5000,strip_accents='unicode',stop_words='english',token_pattern=r'\w{2,}')

In [16]:
tfidf.fit(X_train)
X_train_feat=tfidf.fit_transform(X_train)
print(X_train_feat.shape)

X_val_feat=tfidf.transform(X_val)
print(X_val_feat.shape)

X_test_feat=tfidf.transform(tests_consider)
print(X_test_feat.shape)

(80000, 5000)
(20000, 5000)
(63978, 5000)


In [17]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 2.0MB/s 


In [18]:
print(X_train.shape)
print(y_train.shape)

(80000,)
(80000, 6)


In [18]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

model_br = BinaryRelevance(classifier = SVC(), require_dense = [False, True])
model_br.fit(X_train_feat, y_train)

BinaryRelevance(classifier=SVC(C=1.0, break_ties=False, cache_size=200,
                               class_weight=None, coef0=0.0,
                               decision_function_shape='ovr', degree=3,
                               gamma='scale', kernel='rbf', max_iter=-1,
                               probability=False, random_state=None,
                               shrinking=True, tol=0.001, verbose=False),
                require_dense=[False, True])

In [19]:
preds_train = model_br.predict(X_train_feat)

In [36]:
from sklearn.metrics import roc_auc_score, accuracy_score

print(roc_auc_score(y_train, preds_train.toarray()))

0.7689564894630726


In [ ]:
preds_val = model_br.predict(X_val_feat)


In [37]:
print(roc_auc_score(y_val, preds_val.toarray()))

0.6665239237377513


In [22]:
preds_test = model_br.predict(X_test_feat)

In [38]:
print(roc_auc_score(labels_consider, preds_test.toarray()))

0.6636501540392014


In [19]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC

model_cc = ClassifierChain(classifier = SVC(), require_dense = [False, True])
model_cc.fit(X_train_feat, y_train)

ClassifierChain(classifier=SVC(C=1.0, break_ties=False, cache_size=200,
                               class_weight=None, coef0=0.0,
                               decision_function_shape='ovr', degree=3,
                               gamma='scale', kernel='rbf', max_iter=-1,
                               probability=False, random_state=None,
                               shrinking=True, tol=0.001, verbose=False),
                order=None, require_dense=[False, True])

In [20]:
preds_train = model_cc.predict(X_train_feat)


In [30]:
from sklearn.metrics import roc_auc_score, accuracy_score

print(roc_auc_score(y_train, preds_train.toarray()))

0.7645674561502386


In [ ]:
preds_val = model_cc.predict(X_val_feat)


In [34]:
print(roc_auc_score(y_val, preds_val.toarray()))

0.6751504109269661


In [22]:
preds_test = model_cc.predict(X_test_feat)

In [35]:
print(roc_auc_score(labels_consider, preds_test.toarray()))

0.6718590534519624
